In [2]:
!pip install sqlalchemy clickhouse-connect python-dotenv


[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: C:\Users\villy\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [21]:
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine

# Load environment variables from .env file
load_dotenv(override=True)

def get_postgres_engine():
    try:
        engine = create_engine("postgresql+psycopg2://{user}:{password}@{host}:{port}/{dbname}".format(
                                user=os.getenv('pg_user'),
                                password=os.getenv('pg_password'),
                                host=os.getenv('pg_host'), 
                                port=os.getenv('pg_port'), 
                                dbname=os.getenv('pg_dbname')
                                )
                              )
        with engine.connect() as connection:
            print("Connection to the PostgreSQL database was successful!")
    except Exception as e:
        print("Error: Unable to connect to the PostgreSQL database.")
        print(e)

if __name__ == "__main__":
    get_postgres_engine()


Error: Unable to connect to the PostgreSQL database.
(psycopg2.OperationalError) connection to server on socket "@localhost/.s.PGSQL.5432" failed: No such file or directory
	Is the server running locally and accepting connections on that socket?

(Background on this error at: https://sqlalche.me/e/20/e3q8)
